In [2]:
!pip install langchain


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import os
import json
import requests
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

# Set up your API keys (ensure these are available in your environment)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")
if not NEWSAPI_KEY:
    raise ValueError("Please set the NEWSAPI_KEY environment variable")

# Instantiate the OpenAI LLM (you can adjust temperature and other parameters as needed)
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)



/var/folders/r7/vdpmjwm55fl4_ys4dmdzv2jc0000gn/T/ipykernel_40627/480343525.py:18: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [4]:
# -----------------------------------------------
# Tool 1: Convert user question into SQL-like search query
# -----------------------------------------------
sql_prompt_template = """You are an assistant that converts natural language questions into SQL-like queries for searching news articles.
Assume the articles database has a table named "articles" with a column "content".
For example, if given the question "What is happening with climate change in Europe?" 
you might output: 
    SELECT * FROM articles WHERE content LIKE '%climate change%' AND content LIKE '%Europe%';
    
Now convert the following question into a SQL-like query. Only output the SQL query.

Question: {question}
SQL Query:"""

sql_prompt = PromptTemplate(
    input_variables=["question"],
    template=sql_prompt_template,
)

sql_chain = LLMChain(llm=llm, prompt=sql_prompt, output_key="sql_query")

# -----------------------------------------------
# Tool 2: Run the NewsAPI search using the SQL-like query
# -----------------------------------------------
def search_newsapi(sql_query: str) -> dict:
    """
    A simple function that interprets a SQL-like query by extracting keywords
    and then querying the NewsAPI 'everything' endpoint.
    
    For simplicity, this example extracts words inside LIKE '%...%' clauses.
    A production version would require a robust parser.
    """
    import re
    # Find all substrings inside LIKE '%...%'
    keywords = re.findall(r"LIKE\s+'%([^%]+)%'", sql_query, flags=re.IGNORECASE)
    if not keywords:
        raise ValueError("No keywords extracted from the SQL query.")
    
    # Create a simple query string by joining keywords with AND logic.
    # (NewsAPI uses a simple query parameter where space-separated words are 'AND'ed.)
    query = " ".join(keywords)
    
    print(f"DEBUG: Extracted keywords: {keywords}")
    print(f"DEBUG: Querying NewsAPI with: {query}")
    
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "apiKey": NEWSAPI_KEY,
        "language": "en",
        "pageSize": 5  # Limit the number of articles for brevity
    }
    
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise ValueError(f"NewsAPI request failed: {response.status_code}, {response.text}")
    
    return response.json()

# -----------------------------------------------
# Tool 3: Summarize and format the news articles as Markdown
# -----------------------------------------------
summary_prompt_template = """You are an assistant that summarizes news articles and formats the output as Markdown.
You are provided with the following JSON data (from NewsAPI) that contains news articles.
Please produce a succinct summary that includes:
- A headline summary
- A bullet list of the article titles (with links if available)
- A brief conclusion.

JSON Data:
{news_json}

Your Markdown formatted summary:"""

summary_prompt = PromptTemplate(
    input_variables=["news_json"],
    template=summary_prompt_template,
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="markdown_summary")

# -----------------------------------------------
# Combine everything in a main function
# -----------------------------------------------
def answer_user_question(question: str):
    # 1. Convert the natural language question to a SQL-like query.
    sql_result = sql_chain.run(question=question)
    print("Generated SQL Query:")
    print(sql_result)
    
    # 2. Use the SQL-like query to search NewsAPI.
    news_data = search_newsapi(sql_result)
    # For debugging, you might print the raw JSON (formatted)
    # print(json.dumps(news_data, indent=2))
    
    # 3. Summarize and format the news results as Markdown.
    # We'll convert the news_data dict to a JSON string to pass to the LLM.
    news_json_str = json.dumps(news_data, indent=2)
    markdown_summary = summary_chain.run(news_json=news_json_str)
    
    return markdown_summary


/var/folders/r7/vdpmjwm55fl4_ys4dmdzv2jc0000gn/T/ipykernel_40627/4017695253.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_chain = LLMChain(llm=llm, prompt=sql_prompt, output_key="sql_query")


In [5]:
# -----------------------------------------------
# Example usage
# -----------------------------------------------
if __name__ == "__main__":
    user_question = input("Enter your news-related question: ")
    try:
        markdown_output = answer_user_question(user_question)
        print("\nFinal Markdown Summary:\n")
        print(markdown_output)
    except Exception as e:
        print(f"Error: {e}")

/var/folders/r7/vdpmjwm55fl4_ys4dmdzv2jc0000gn/T/ipykernel_40627/4017695253.py:87: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sql_result = sql_chain.run(question=question)


Generated SQL Query:
 SELECT * FROM articles WHERE content LIKE '%trump%' AND content LIKE '%gaza%';
DEBUG: Extracted keywords: ['trump', 'gaza']
DEBUG: Querying NewsAPI with: trump gaza

Final Markdown Summary:



# Headline Summary:
Trump proposes Egypt and Jordan take in Palestinians from Gaza, sparking controversy and accusations of ethnic cleansing.

# Article Titles:
- [Trump says he wants Egypt and Jordan to take in Palestinians from Gaza](https://www.bbc.com/news/articles/c07kpjyzgllo)
- [Trump's Gaza plan will be seen as flying in face of international law](https://www.bbc.com/news/articles/c9w5q8qn59yo)
- [Anthony Zurcher: Real-estate instincts clash with America First worldview](https://www.bbc.com/news/articles/ce9nmm3nvdno)
- [Israel has changed since Donald Trump's last term – has he?](https://www.bbc.com/news/articles/cgkjxlml42vo)
- ['A long, long road ahead': Gaza rebuilds from zero](https://www.bbc.com/news/articles/cx2p916dngqo)

# Conclusion:
President Trump's propo